In [1]:
import tkinter as tk

from PIL import Image, ImageTk

import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import cv2

from tkinter import filedialog


root = tk.Tk()
root.title("Color Detector")
root.geometry("650x530")

my_frame = tk.Frame(root, width=650, height=530) 
my_frame.pack()

knc = KNeighborsClassifier(n_neighbors=5)
svc = SVC(kernel='linear')

filename=""

def createModel():
    name,r,g,b,rgb=[],[],[],[],[]
    
    skip=0
    with open("wikipedia_color_names.csv") as colorsFile:
        csvReader=csv.reader(colorsFile)
        for i in csvReader:
            if(skip!=0):
                #print(i[0],i[2],i[3],i[4])
                name.append(i[0])
                r.append(int(i[2]))
                g.append(int(i[3]))
                b.append(int(i[4]))
            else:
                skip=1
                continue
        
    rgb=list(zip(r,g,b))
    #print(rgb)
    #print(name,rgb)
    
    
    knc.fit(rgb, name)
    
    svc.fit(rgb, name)
    

def predictColor(filepath):
    #print("file sent", filepath)
    image = cv2.imread(filepath)
    rsImg = cv2.resize(image, (340,480))

    #cv2.imshow("red_img", rsImg)
    #cv2.waitKey(0)
    #cv2.destroyWindow("red_img")

    colorbgr = rsImg[10,10]  ### middle of the image
    #print(colorbgr)

    colorrgb=colorbgr[::-1]
    #print(colorrgb)

    kncprediction=knc.predict([colorrgb])
    #print(kncprediction)
    l11 = tk.Label(root,font=8, text=kncprediction[0]).place(x=50, y=380)

    svcprediction=svc.predict([colorrgb])
    #print(svcprediction)
    l11 = tk.Label(root,font=8, text=svcprediction[0]).place(x=350, y=380)
    

def chooseFile():
    global filename 
    filename = filedialog.askopenfilename(parent=root, initialdir= "/", title='Please select a directory')
    #print("file selected", filename)
    
    im = Image.open(filename)
    tkimage = ImageTk.PhotoImage(im)
    myvar=tk.Label(root, width=150, height=150, image = tkimage)
    myvar.image = tkimage
    myvar.place(x=250, y=100)
   

l1 = tk.Label(root,font=8, text="KNN Prediction:-").place(x=50, y=350)
l2 = tk.Label(root,font=8, text="SVM Prediction:-").place(x=350, y=350)
l3 = tk.Label(root,font=8, text="Color Predictor").place(x=255, y=10)
credit = tk.Label(root,font=16, text="Created by:- Prashant Dhiman, Prateek, Sahil\nDCRUST, Sonipat").place(x=120, y=460)

b1 = tk.Button(root,font=8, text="Create Model", command=lambda: createModel()).place(x=50, y=300)
b2 = tk.Button(root,font=8, text="Predict Color", command=lambda : predictColor(filename)).place(x=200, y=300)
b3 = tk.Button(root,font=8, text="Browse Image", command=lambda : chooseFile()).place(x=350, y=300)
b4 = tk.Button(root,font=8, text="Close", command=root.destroy).place(x=520, y=300)

root.mainloop()